# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096898


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:46,  3.66s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:14,  2.65s/it]

Rendering models:  13%|█▎        | 4/31 [00:08<00:54,  2.00s/it]

Rendering models:  19%|█▉        | 6/31 [00:08<00:35,  1.44s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:12<00:00,  4.58it/s]

Rendering models:  94%|█████████▎| 29/31 [00:13<00:00,  4.68it/s]

Rendering models:  97%|█████████▋| 30/31 [00:13<00:00,  4.39it/s]

Rendering models: 100%|██████████| 31/31 [00:13<00:00,  4.54it/s]

not-logged-in-461bf2f260fc99e4866e    0.049784
not-logged-in-964f4e4ef2fd71d35dfb    0.000214
Nid519                                0.224641
awright5                              0.001933
thiagoxxxx                            0.000258
jaksonA                               0.041333
not-logged-in-a6ae1cef5bd4f44a5fa9    0.000634
rmz4160                               0.000513
Mckay_Oyler                           0.000176
betelgeeuse84                         0.010682
Giantjack14                           0.013948
ElisabethB                            0.000177
pangeli5                              0.000569
arad98                                0.000304
surfgimp                              0.056508
Lavadude                              0.004886
helios1235                            0.000199
ShirleyL                              0.000286
jmacd297                              0.000784
Alissa18                              0.000367
not-logged-in-3429675089c67c4fccb8    0.040008
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())